In [1]:
import pandas as pd
import numpy as np
import pickle
import os

In [2]:
with open('movie_1024_embeddings.pkl', 'rb') as f:
    movie_1024_embeddings = pickle.load(f)
print(len(movie_1024_embeddings), len(movie_1024_embeddings[204]))

3883 1024


In [3]:
user_movie_dict_path = '../train_valid_test/user_movie_dict.pkl'
with open(user_movie_dict_path, 'rb') as f:
    user_movie_dict = pickle.load(f)
print(len(user_movie_dict), user_movie_dict[1])

6040 [3186, 1721, 1022, 1270, 2340, 1836, 3408, 1207, 2804, 720, 1193, 260, 919, 608, 2692, 1961, 2028, 3105, 938, 1962, 1035, 150, 2018, 1028, 1097, 914, 2797, 1287, 2762, 1246, 661, 2918, 531, 3114, 2791, 1029, 2321, 1197, 594, 2398, 1545, 527, 1, 588, 2687, 745, 595, 2294, 783, 2355, 1907, 1566, 48]


In [4]:
user_valid_path = '../train_valid_test/user_valid.pkl'
with open(user_valid_path, 'rb') as f:
    user_valid_dict = pickle.load(f)
print(len(user_valid_dict), user_valid_dict[1])

6040 [1566]


In [5]:
user_test_path = '../train_valid_test/user_test.pkl'
with open(user_test_path, 'rb') as f:
    user_test_dict = pickle.load(f)
print(len(user_test_dict), user_test_dict[1])

6040 [48]


In [9]:
ratings_df = pd.read_csv('../ratings.csv')
ratings_df.head()

,user_id,movie_id,rating
0,1,3186,4
1,1,1721,4
2,1,1022,5
3,1,1270,5
4,1,2340,3


In [21]:
from numpy.linalg import norm

def findKSimilarMovies(user, valid_movie, k):
    movie_sim_dict = dict()
    # movies_watched = user_movie_dict[user][:-2] ### For Valid movie
    movies_watched = user_movie_dict[user][:-1] ### For Test movie
    # print("movies_watched:", movies_watched)
    valid_movie_norm = norm(movie_1024_embeddings[valid_movie])
    for movie in movies_watched:
        movie_sim_dict[movie] = np.dot(movie_1024_embeddings[valid_movie], movie_1024_embeddings[movie])/(valid_movie_norm * norm(movie_1024_embeddings[movie]))
    movie_sim_dict = dict(sorted(movie_sim_dict.items(), key = lambda item: item[1], reverse = True))
    return list(movie_sim_dict.keys())[:20]

In [8]:
k = 20
user_valid_sem_sim_movies = dict()
for user, movie in user_valid_dict.items():
    # print(user, movie)
    k_similar_movie_list = findKSimilarMovies(user, movie[0], k)
    # print(len(k_similar_movie_list), k_similar_movie_list)
    user_valid_sem_sim_movies[user] = k_similar_movie_list
    # break
print(len(user_valid_sem_sim_movies), user_valid_sem_sim_movies[1])

6040 [595, 588, 783, 594, 2687, 1022, 1029, 1907, 1, 3114, 919, 2355, 2018, 2294, 1197, 661, 1028, 2321, 938, 1270]


In [19]:
user_valid_sem_sim_data = dict()
for user, sem_sim_movies in user_valid_sem_sim_movies.items():
    cnt += 1
    user_items = ratings_df[ratings_df['user_id'] == user]['movie_id'].to_list()
    user_items = [item for item in user_items if item in sem_sim_movies]
    user_valid_sem_sim_data[user] = user_items
print(len(user_valid_sem_sim_data), user_valid_sem_sim_data[1])

6040 [1022, 1270, 919, 938, 2018, 1028, 661, 3114, 1029, 2321, 1197, 594, 1, 588, 2687, 595, 2294, 783, 2355, 1907]


In [20]:
with open('./sem_sim_dataset/sem_sim_valid.pkl', 'wb') as f:
    pickle.dump(user_valid_sem_sim_data, f)

In [23]:
k = 20
user_test_sem_sim_movies = dict()
for user, movie in user_test_dict.items():
    # print(user, movie)
    k_similar_movie_list = findKSimilarMovies(user, movie[0], k)
    # print(len(k_similar_movie_list), k_similar_movie_list)
    user_test_sem_sim_movies[user] = k_similar_movie_list
    # break
print(len(user_test_sem_sim_movies), user_test_sem_sim_movies[1])

6040 [595, 783, 2687, 588, 1022, 2018, 1029, 1907, 1566, 661, 594, 1721, 1, 1197, 531, 1028, 919, 3114, 1246, 2321]


In [24]:
user_test_sem_sim_data = dict()
for user, sem_sim_movies in user_test_sem_sim_movies.items():
    cnt += 1
    user_items = ratings_df[ratings_df['user_id'] == user]['movie_id'].to_list()
    user_items = [item for item in user_items if item in sem_sim_movies]
    user_test_sem_sim_data[user] = user_items
print(len(user_test_sem_sim_data), user_test_sem_sim_data[1])

6040 [1721, 1022, 919, 2018, 1028, 1246, 661, 531, 3114, 1029, 2321, 1197, 594, 1, 588, 2687, 595, 783, 1907, 1566]


In [25]:
with open('./sem_sim_dataset/sem_sim_test.pkl', 'wb') as f:
    pickle.dump(user_test_sem_sim_data, f)